# Example 1
https://github.com/roman-kh/roman-kh.github.com/blob/master/files/linear-models/slm.ipynb

In [7]:
import numpy as np
import patsy as pt
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
%matplotlib inline
import sklearn.linear_model as lm
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 12, 10

In [3]:
# load a data file
data = pd.read_csv('/home/akimg/Projects/ML_NeighborsClassifier-master/heart.csv', sep=',')
# x - a table with factors x1,x2,x3
# y - a table with a dependent variable
x = data.iloc[:,:-1]
y = data.iloc[:,-1]

data.head(5)

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [4]:
# запускаем расчет параметров для указанных данных
# и выведем параметры рассчитанной модели
skm = lm.LinearRegression()
skm.fit(x, y)
skm.intercept_, skm.coef_

(0.8288986814219779,
 array([-0.00082042, -0.19599555,  0.11270339, -0.00199097, -0.00035354,
         0.01737364,  0.04984795,  0.00301933, -0.14404593, -0.05878869,
         0.07897881, -0.10060216, -0.11903917]))

In [5]:
# add constant to calculate intercept
x_ = sm.add_constant(x)
smm = sm.OLS(y, x_)
res = smm.fit()
b = res.params.values

b

/home/akimg/.local/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2495: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


array([ 8.28898681e-01, -8.20415178e-04, -1.95995553e-01,  1.12703391e-01,
       -1.99096545e-03, -3.53538744e-04,  1.73736362e-02,  4.98479529e-02,
        3.01932913e-03, -1.44045931e-01, -5.87886904e-02,  7.89788053e-02,
       -1.00602157e-01, -1.19039167e-01])

In [6]:
# create a model with a formula
smm = sm.OLS.from_formula("target ~ thal + ca + exang", data=data)
# run calculations
res = smm.fit()

print(res.params.values)

[ 1.19992621 -0.18144798 -0.15460276 -0.37591311]


In [7]:
# create a factor matrix and a result matrix from a dataframe
pt_y, pt_x = pt.dmatrices("target ~ thal + ca + exang", data=data)
# use ordinary least squares
res = np.linalg.lstsq(pt_x, pt_y)
# show the model coefficients
res[0].ravel()

/usr/lib/python3/dist-packages/ipykernel_launcher.py:4: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  after removing the cwd from sys.path.


array([ 1.19992621, -0.18144798, -0.15460276, -0.37591311])

In [7]:
import ggplot
ggplot(aes(x="thalach", y="target"), data=data) + geom_point()

AttributeError: module 'pandas' has no attribute 'tslib'

# Example 2
(L3)

In [8]:
i = 0
for col in data.columns:
    if str(col) == 'target':
        colname = 'y'
    else:
        colname = 'x_%d'%i
    data[colname] = data[col]**i
    data.drop(col, axis='columns', inplace=True)
    i += 1

data.head()

,x_0,x_1,x_2,x_3,x_4,x_5,x_6,x_7,x_8,x_9,x_10,x_11,x_12,y
0,1,1,9,3048625,2947295521,1,0,1708593750000000,0,1801.152661,0,0,1,1
1,1,1,4,2197000,3906250000,0,1,7996339888664083,0,78815.638672,0,0,4096,1
2,1,0,1,2197000,1731891456,0,0,4453476124377088,0,20.661047,1024,0,4096,1
3,1,1,1,1728000,3102044416,0,1,5661610866627712,0,0.134218,1024,0,4096,1
4,1,0,0,1728000,15704099856,0,1,3057125241215467,1,0.010078,1024,0,4096,1


In [9]:
#Import Linear Regression model from scikit-learn.
from sklearn.linear_model import LinearRegression
def linear_regression(data, power, models_to_plot):
    #initialize predictors:
    predictors=['x']
    if power>=2:
        predictors.extend(['x_%d'%i for i in range(2,power+1)])
    
    #Fit the model
    linreg = LinearRegression(normalize=True)
    linreg.fit(data[predictors],data['y'])
    y_pred = linreg.predict(data[predictors])
    
    #Check if a plot is to be made for the entered power
    if power in models_to_plot:
        plt.subplot(models_to_plot[power])
        plt.tight_layout()
        plt.plot(data['x'],y_pred)
        plt.plot(data['x'],data['y'],'.')
        plt.title('Plot for power: %d'%power)
    
    #Return the result in pre-defined format
    rss = sum((y_pred-data['y'])**2)
    ret = [rss]
    ret.extend([linreg.intercept_])
    ret.extend(linreg.coef_)
    return ret

In [10]:
col = ['x_%d'%i for i in range(0,15)]
ind = ['model_pow_%d'%i for i in range(0,15)]

coef_matrix_simple = pd.DataFrame(index=ind, columns=col)

#Define the powers for which a plot is required:
models_to_plot = {1:231,3:232,6:233,9:234,12:235,15:236}

#Iterate through all powers and assimilate results
for i in range(0,15):
    coef_matrix_simple.iloc[i-1,0:i+2] = linear_regression(data, power=i, models_to_plot=models_to_plot)


KeyError: "None of [Index(['x'], dtype='object')] are in the [columns]"

# Example 3
https://coderlessons.com/tutorials/python-technologies/logisticheskaia-regressiia-v-python/logisticheskaia-regressiia-v-python-kratkoe-rukovodstvo

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [12]:
data = pd.read_csv('/home/akimg/Projects/ML_NeighborsClassifier-master/heart.csv', sep=',')
data.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [13]:
data = pd.get_dummies(data, columns=data.columns)
data.head(5)

,age_29,age_34,age_35,age_37,age_38,age_39,age_40,age_41,age_42,age_43,...,ca_1,ca_2,ca_3,ca_4,thal_0,thal_1,thal_2,thal_3,target_0,target_1
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
1,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1
2,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,1
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1


In [14]:
X = data.iloc[:,1:]
Y = data.iloc[:,0]
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=0)

In [15]:
classifier = LogisticRegression(solver='lbfgs',random_state=0)
classifier.fit(X_train, Y_train)
LogisticRegression(C = 1.0, class_weight = None, dual = False, 
                   fit_intercept=True, intercept_scaling=1, max_iter=100, 
                   multi_class='target', n_jobs=None, penalty='l2', random_state=0,
                   solver='lbfgs', tol=0.0001, verbose=0, warm_start=False)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='target', n_jobs=None, penalty='l2',
                   random_state=0, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [16]:
predicted_y = classifier.predict(X_test)
for x in range(len(predicted_y)):
    if (predicted_y[x] == 1):
        print(x, end="\t")

In [17]:
print('Accuracy: {:.2f}'.format(classifier.score(X_test, Y_test)))


Accuracy: 1.00


# Example 4
https://towardsdatascience.com/building-a-logistic-regression-in-python-step-by-step-becd4d56c9c8

In [18]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
import matplotlib.pyplot as plt 
plt.rc("font", size=14)
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import seaborn as sns
sns.set(style="white")
sns.set(style="whitegrid", color_codes=True)

In [19]:
data = pd.read_csv('/home/akimg/Projects/ML_NeighborsClassifier-master/heart.csv', sep=',')
data.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [20]:
count_no_sub = len(data[data['y']==0])
count_sub = len(data[data['y']==1])
pct_of_no_sub = count_no_sub/(count_no_sub+count_sub)
print("percentage of no subscription is", pct_of_no_sub*100)
pct_of_sub = count_sub/(count_no_sub+count_sub)
print("percentage of subscription", pct_of_sub*100)

KeyError: 'y'

# Example 5
(L3)

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.linear_model import LogisticRegression

In [9]:
data = pd.read_csv('/home/akimg/Projects/ML_NeighborsClassifier-master/heart.csv', sep=',')
data.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [10]:
X = data[data.columns[:-1]]
y = data['target']
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [11]:
classifier = LogisticRegression()
classifier.fit(X, y)

/home/akimg/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [13]:
def plot_boundary(clf, X, y):
    xx, yy = np.meshgrid(np.linspace(-3, 3, 50), np.linspace(-3, 3, 50))
    Z = clf.predict_proba(np.vstack((xx.ravel(), yy.ravel())).T)[:, 1].reshape(xx.shape)
    image = plt.imshow(Z, interpolation='nearest',
                       extent=(xx.min(), xx.max(), yy.min(), yy.max()),
                       aspect='auto', origin='lower', cmap=plt.cm.PuOr_r)
#     contours = plt.contour(xx, yy, Z, levels=[0], linewidths=2, linetypes='--')
#     plt.scatter(X[:, 0], X[:, 1], s=30, c=y, cmap=plt.cm.Paired)
#     plt.axis([-3, 3, -3, 3])
#     plt.colorbar(image)
    
plot_boundary(classifier, X, y)

ValueError: X has 2 features per sample; expecting 13

In [14]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
new_classifier = Pipeline([('poly', PolynomialFeatures(degree=2)), ('logit', LogisticRegression())])
new_classifier.fit(X, y)
# Обучите новый классификатор на тех же данных и посмотрим, как он себя ведет в исходном пространстве признаков.

plot_boundary(new_classifier, X, y)

/home/akimg/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/akimg/.local/lib/python3.6/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


ValueError: X shape does not match training shape

In [27]:
X_test = rng.randn(300, 2)
y_test = np.logical_xor(X_test[:, 0] < 0, X_test[:, 1] < 0)

new_classifier.score(X_test, y_test)

NameError: name 'rng' is not defined